<a href="https://colab.research.google.com/github/tibocour/IA/blob/main/train_tflite_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilisation des scripts

Ce notebook vise à montrer l'utilisation des scripts du projet afin d'effectuer un entrainement et une inférence.

In [1]:
%%shell

nvidia-smi

Fri Feb 24 11:22:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    47W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installation du projet `bfc` et accès aux données

Les login et mot de passe sont nécessaires pour accéder à des repos privés à partir de Google/Colab. Cela ne serait pas le cas en accès public.

In [36]:
import os
from getpass import getpass
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password

GitHub user··········
GitHub password··········


In [4]:
%%shell

git clone https://$GITHUB_AUTH@github.com/tibocour/IA.git

Cloning into 'IA'...
remote: Enumerating objects: 227, done.
remote: Total 227 (delta 0), reused 0 (delta 0), pack-reused 227
Receiving objects: 100% (227/227), 119.17 MiB | 47.78 MiB/s, done.
Resolving deltas: 100% (121/121), done.


# Installation des dépendances

In [5]:
%%shell

pip install -r IA/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.8/577.8 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 94.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 KB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 96.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.

# Apprentissage par transfer learning

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
%%shell
cp -r /content/drive/MyDrive/label_studio /content/IA/data

In [10]:
%%shell

python IA/python/train.py \
  --train_zip IA/data/label_studio/train_label-studio.zip \
  --valid_zip IA/data/label_studio/valid_label-studio.zip 

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
Tensorflow version : 2.8.4
args : Namespace(batch_size=10, epochs=50, train_zip='IA/data/label_studio/train_label-studio.zip', valid_zip='IA/data/label_studio/valid_label-studio.zip', with_tflite_eval=0)
created zip temporary directory: /tmp/tmpdqar7fzh
extract zip file IA/data/la

# Inférence


In [70]:
%%shell

rm -rf /tmp/images /tmp/annotations
unzip -q IA/data/label_studio/valid_roboflow.zip -d /tmp

ls /tmp/images

0001.jpg  0223.jpg  0456.jpg  0668.jpg	0874.jpg  1118.jpg  1328.jpg  1526.jpg
0003.jpg  0224.jpg  0461.jpg  0669.jpg	0877.jpg  1119.jpg  1330.jpg  1531.jpg
0004.jpg  0239.jpg  0465.jpg  0670.jpg	0879.jpg  1121.jpg  1331.jpg  1536.jpg
0007.jpg  0241.jpg  0466.jpg  0674.jpg	0882.jpg  1122.jpg  1334.jpg  1543.jpg
0009.jpg  0244.jpg  0469.jpg  0677.jpg	0885.jpg  1123.jpg  1337.jpg  1549.jpg
0018.jpg  0246.jpg  0471.jpg  0690.jpg	0889.jpg  1125.jpg  1347.jpg  1551.jpg
0019.jpg  0250.jpg  0490.jpg  0693.jpg	0898.jpg  1135.jpg  1350.jpg  1552.jpg
0021.jpg  0252.jpg  0502.jpg  0694.jpg	0899.jpg  1139.jpg  1362.jpg  1555.jpg
0024.jpg  0266.jpg  0514.jpg  0696.jpg	0901.jpg  1140.jpg  1364.jpg  1556.jpg
0029.jpg  0268.jpg  0520.jpg  0701.jpg	0907.jpg  1151.jpg  1367.jpg  1561.jpg
0045.jpg  0271.jpg  0524.jpg  0704.jpg	0916.jpg  1158.jpg  1371.jpg  1575.jpg
0047.jpg  0284.jpg  0528.jpg  0707.jpg	0922.jpg  1167.jpg  1372.jpg  1577.jpg
0051.jpg  0287.jpg  0530.jpg  0710.jpg	0925.jpg  1168.jpg  1373.

In [71]:
%%shell

# Change to the directory that contains the script
cd /content/IA/python/google-coral-inference/

# Create a new directory to save the processed images
output_dir="/content/valid_processed"
mkdir -p "$output_dir"

# Loop through all the images in the input directory
for filename in /tmp/images/*.jpg; do
  # Get the input and output paths
  input_path="$filename"
  output_path="$output_dir/$(basename "$filename")"
  
  # Run object detection on the image
  python3 detect_image.py \
    --model /content/efficientdet-lite-bfc.tflite \
    --labels /content/bfc-labels.txt \
    --input "$input_path" \
    --output "$output_path"
done


With Edge TPU ? False
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
----INFERENCE TIME----
Note: The first inference is slow because it includes loading the model into Edge TPU memory.
2605.43 ms
2816.49 ms
2604.09 ms
2602.35 ms
2600.49 ms
-------RESULTS--------
bouchon
  id:     0
  score:  0.96875
  bbox:   BBox(xmin=85, ymin=32, xmax=495, ymax=420)
bouchon
  id:     0
  score:  0.84375
  bbox:   BBox(xmin=249, ymin=469, xmax=648, ymax=644)
bouchon
  id:     0
  score:  0.7734375
  bbox:   BBox(xmin=313, ymin=105, xmax=630, ymax=504)
bouchon
  id:     0
  score:  0.6875
  bbox:   BBox(xmin=-6, ymin=457, xmax=509, ymax=635)
With Edge TPU ? False
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
----INFERENCE TIME----
Note: The first inference is slow because it includes loading the model into Edge TPU memory.
2606.56 ms
2591.64 ms
2600.41 ms
2811.63 ms
2603.40 ms
-------RESULTS--------
bouchon
  id:     0
  score:  0.9765625
  bbox:   BBox(xmin=113, ymin=104, xmax=455,

CalledProcessError: ignored

In [61]:
%%shell
folder_name="valid_processed"

# Créer le fichier zip
python -c "import shutil; shutil.make_archive('$folder_name', 'zip', '$folder_name')"


In [62]:
from google.colab import files
files.download("efficientdet-lite-bfc.tflite")
files.download("bfc-labels.txt")
files.download("valid_processed.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>